Just getting started with LAVIS, delete this at some point!

Checking if random crops of varying sizes of ImageNet-images can even be described well, or if we just get garbage out...

In [ ]:
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
from PIL import Image

from lavis.models import load_model_and_preprocess

# setup device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
IMAGENET_PATH = "/mnt/qb/datasets/ImageNet2012/"

def get_random_crops(target_dir, n_batches):

    os.makedirs(target_dir, exist_ok=False)

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomCrop(128),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    testset = torchvision.datasets.ImageNet(root=IMAGENET_PATH, split='val', transform=test_transform)
 
    testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True, pin_memory=True, num_workers=8)

    topil = transforms.ToPILImage()

    idx = 0
    for imgs, _ in testloader:

        for i in range(32):
            img = topil(imgs[i,:,:,:])
            img.save(os.path.join(target_dir, str(i)+".png"))

        idx += 1
        if idx >= n_batches:
            break

get_random_crops("test_images_128", 1)


In [ ]:
# loads BLIP caption base model, with finetuned checkpoints on MSCOCO captioning dataset.
# this also loads the associated image processors
model, vis_processors, _ = load_model_and_preprocess(name="blip_caption", model_type="base_coco", is_eval=True, device=device)


In [ ]:
# load sample image
# raw_image = Image.open("docs/_static/merlion.png").convert("RGB")

def caption_img(img_path):

    raw_image = Image.open(img_path).convert("RGB")
    # preprocess the image
    # vis_processors stores image transforms for "train" and "eval" (validation / testing / inference)
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

    # generate caption
    return model.generate({"image": image}), raw_image
    # ['a large fountain spewing water into the air']

In [ ]:
import os

def caption_dir(dir):
    root, dirs, files = next(os.walk(dir))
    res = {}
    for file in files:
        caption, raw_image = caption_img(os.path.join(root, file))
        res[file[:-4]] = caption, raw_image

    return res

In [ ]:
import matplotlib.pyplot as plt 

img_dir = "test_images_128"
out_dir = img_dir + "_output"

os.makedirs(out_dir, exist_ok=True)

captions = caption_dir(img_dir)

for name, (caption, img) in captions.items():

    fig = plt.figure()
    plt.title(caption[0])
    fig.set_facecolor("white")
    plt.imshow(img)
    plt.savefig(os.path.join(out_dir, name+".png"), transparent=False)
    plt.show()
    plt.close()


In [ ]:
print(captions["4"])